# Pybind11 test for ControlPlane Submodule

First import the library "e2sar_py". Make sure the module is included in the path.

In [1]:
import sys

## IMPORTANT: Update the path to your built Python module
sys.path.append(
    '/home/xmei/Documents/hpdf_projects/E2SAR/build/src/pybind')

import e2sar_py

# Print to check the attributes
# print(dir(e2sar_py))

In [2]:
print(dir(e2sar_py.ControlPlane))

['LBManager', 'LBStatus', 'LBWorkerStatus', 'LoadBalancerStatusReply', '__doc__', '__loader__', '__name__', '__package__', '__spec__']


## "LBManager" class
Intialize `LBManager`.

In [3]:
print(dir(e2sar_py.ControlPlane.LBManager))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'deregister_worker', 'free_lb', 'free_lb_by_id', 'get_lb', 'get_lb_by_id', 'get_lb_status', 'get_lb_status_by_id', 'get_port_range', 'get_uri', 'get_version', 'make_ssl_options', 'register_worker', 'reserve_lb_seconds', 'send_state']


In [4]:
TEST_URI = "ejfat://token@192.188.29.6:18020/lb/36?sync=192.188.29.6:19020&data=192.188.29.20"

uri = e2sar_py.EjfatURI(TEST_URI)

lb_manager = e2sar_py.ControlPlane.LBManager(uri)

In [5]:
res_uri = lb_manager.get_uri()  # return an EjfatURI object

print("Has data address: ", res_uri.has_data_addr())
print("Has sync address: ", res_uri.has_sync_addr())

res_cp_addr = res_uri.get_cp_addr().value()
print("Control plane address: ", res_cp_addr[0], ":", res_cp_addr[1])

res_data_addr_v4 = res_uri.get_data_addr_v4().value()
print("Data plane address (IPv4): ", res_data_addr_v4[0], ":", res_data_addr_v4[1])

print("Admin token: ", res_uri.get_admin_token().value())

Has data address:  True
Has sync address:  True
Control plane address:  192.188.29.6 : 18020
Data plane address (IPv4):  192.188.29.20 : 19522
Admin token:  token


### Tests with a mock EJ-FAT Load Balancer

Below contexts need a real LB running

#### Reserve a LB with seconds

The Python `datetime.timedelta` package is very convenient to get durations. For simplicity, we create a C++ method `LBManager::reserveLB` based on duration in seconds.

In [6]:
from datetime import timedelta

duration_seconds = timedelta(hours=1).total_seconds()
ip_list = ["192.168.100.1", "192.168.100.2"]

try:
    reserve_result = lb_manager.reserve_lb_seconds(
        lb_id="ejfat", seconds=duration_seconds, senders=ip_list)
    print(reserve_result.value())
except RuntimeError as e:
    print("Caught runtime error:", e)

Caught runtime error: Error connecting to LB CP in reserveLB(): failed to connect to all addresses; last error: UNKNOWN: ipv4:192.188.29.6:18020: Failed to connect to remote host: FD Shutdown


## "LBWorkerStatus" class

In [9]:
# %pip install protobuf

'''Cannot direct map Python protobuf TimeStamp to C++ protobuf TimeStamp So we created our own.'''

from google.protobuf.timestamp_pb2 import Timestamp as gts
from e2sar_py import Timestamp

# The protobuf::TimeStamp object
g_ts = gts()
g_ts.GetCurrentTime()
print(g_ts)


seconds: 1724491121
nanos: 843828000



In [10]:

# Create a e2sar Timestamp object
curr_ts = Timestamp()
print(dir(curr_ts))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'get_nanos', 'get_seconds', 'set_nanos', 'set_seconds']


In [11]:
curr_ts.set_seconds(g_ts.seconds)
curr_ts.set_nanos(g_ts.nanos)

print(f"Timestamp: seconds = {curr_ts.get_seconds()}, nanos = {curr_ts.get_nanos()}")

Timestamp: seconds = 1724491121, nanos = 843828000


In [12]:
print(dir(e2sar_py.ControlPlane.LBWorkerStatus))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'control_signal', 'fill_percent', 'last_updated', 'name', 'slots_assigned']


In [13]:
worker1 = e2sar_py.ControlPlane.LBWorkerStatus("Worker1", 75.5, 0.9, 10, curr_ts)

In [ ]:
# Create LBWorkerStatus objects
# worker1 = e2sar_py.ControlPlane.LBWorkerStatus("Worker1", 75.5, 0.9, 10, timestamp)
# worker2 = e2sar_py.ControlPlane.LBWorkerStatus("Worker2", 80.2, 1.1, 15, timestamp)

# workers = [worker1, worker2]
# sender_addresses = ["address1", "address2"]

# # Create an LBStatus object
# status = e2sar_py.ControlPlane.LBStatus(timestamp, 1234, 5678, workers, sender_addresses, timestamp)

# # Access members
# print(status.currentEpoch)
# print(status.senderAddresses)
# print([worker.name for worker in status.workers])